In [1]:
!nvidia-smi


Sat Dec  7 12:00:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080        Off |   00000000:0A:00.0 Off |                  N/A |
| 33%   46C    P8             31W /  225W |      41MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import jax.numpy as jnp


In [3]:
data = jnp.load('output/ant_omni/mcpg_me/2024-11-18_114515_007801/repertoire/genotypes.npy')

In [6]:
jnp.min(data)

Array(-1.3110462, dtype=float32)

In [3]:
import functools
import pickle

import jax
import jax.numpy as jnp
from flax import serialization

from qdax import environments
from qdax.tasks.brax_envs import reset_based_scoring_function_brax_envs as scoring_function
from qdax.core.neuroevolution.buffers.buffer import PPOTransition, QDTransition, QDMCTransition
from qdax.core.neuroevolution.networks.networks import MLPMCPG, MLP, MLPDC, MLPMCPG_

from brax.v1.io import html
from IPython.display import HTML
from omegaconf import OmegaConf
from utils import get_env, get_config


In [4]:
genotypes = jnp.load("output/hopper_uni_250/mcpg_me_fixed/2024-11-27_141857_414364")

IsADirectoryError: [Errno 21] Is a directory: 'output/hopper_uni_250/mcpg_me_fixed/2024-11-27_141857_414364'

In [8]:
jnp.min(genotypes)

Array(-1.4786335, dtype=float32)

In [4]:
from pathlib import Path


run_dir = Path("final_tuning_1000/output/hopper_uni_1000/mcpg_me_fixed/2024-12-07_050738_695012")
config = get_config(run_dir)



In [5]:
rng = jax.random.PRNGKey(config.seed)
env = get_env(config)
reset_fn = jax.jit(env.reset)
step_fn = jax.jit(env.step)


policy_network = MLPMCPG_(
    action_dim=env.action_size,
    activation=config.algo.activation,
    no_neurons=config.algo.no_neurons,
)

#policy_layer_sizes = config.policy_hidden_layer_sizes + (env.action_size,)
#policy_network = MLP(
#    layer_sizes=policy_layer_sizes,
#    kernel_init=jax.nn.initializers.lecun_uniform(),
#    final_activation=jnp.tanh,
#)
#actor_dc_network = MLPDC(
#    layer_sizes=policy_layer_sizes,
#    kernel_init=jax.nn.initializers.lecun_uniform(),
#    final_activation=jnp.tanh,
#)

    
@jax.jit
def play_step_fn(env_state, policy_params, random_key):
    #random_key, subkey = jax.random.split(random_key)
    pi, action = policy_network.apply(policy_params, env_state.obs)
    logp = pi.log_prob(action)
    #logp = policy_network.apply(policy_params, env_state.obs, actions, method=policy_network.logp)
    state_desc = env_state.info["state_descriptor"]
    next_state = env.step(env_state, action)

    transition = QDMCTransition(
        obs=env_state.obs,
        next_obs=next_state.obs,
        rewards=next_state.reward,
        dones=next_state.done,
        truncations=next_state.info["truncation"],
        actions=action,
        state_desc=state_desc,
        next_state_desc=next_state.info["state_descriptor"],
        logp=logp,
    )

    return (next_state, policy_params, random_key), transition


#@jax.jit
#def play_step_fn(env_state, policy_params, random_key):
#    actions = policy_network.apply(policy_params, env_state.obs)
#    state_desc = env_state.info["state_descriptor"]
#    next_state = env.step(env_state, actions)

#    transition = QDTransition(
#        obs=env_state.obs,
#        next_obs=next_state.obs,
#        rewards=next_state.reward,
#        dones=next_state.done,
#        truncations=next_state.info["truncation"],
#        actions=actions,
#        state_desc=state_desc,
#        desc=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
#        next_state_desc=next_state.info["state_descriptor"],
#        desc_prime=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
#    )

#    return (next_state, policy_params, random_key), transition



bd_extraction_fn = environments.behavior_descriptor_extractor[config.env.name]
scoring_fn = functools.partial(
    scoring_function,
    episode_length=config.env.episode_length,
    play_reset_fn=reset_fn,
    play_step_fn=play_step_fn,
    behavior_descriptor_extractor=bd_extraction_fn,
)

# Build the reconstruction function
fake_obs = jnp.zeros(shape=(env.observation_size,))
fake_desc = jnp.zeros(shape=(env.behavior_descriptor_length,))

In [6]:
from jax.flatten_util import ravel_pytree
from qdax.core.containers.mapelites_repertoire import MapElitesRepertoire


random_key, random_subkey = jax.random.split(rng)
fake_params = policy_network.init(random_subkey, fake_obs)

_, reconstruction_fn = ravel_pytree(fake_params)

# Build the repertoire
repertoire = MapElitesRepertoire.load(reconstruction_fn=reconstruction_fn, path=str(run_dir) + "/repertoire/")

In [8]:
repertoire.descriptors

Array([[0.45800003],
       [0.777666  ],
       [0.163     ],
       ...,
       [0.09716599],
       [0.12857144],
       [0.855     ]], dtype=float32)

Array([False, False, False, ..., False, False, False], dtype=bool)

In [7]:
#indices = [index for index, (x) in enumerate(repertoire.descriptors) if 0 < x < 0.2]
indices = [index for index, x in enumerate(repertoire.fitnesses) if    x > 2000]

#indices__ = [i for i in indices if i in indices_]
print("Requests indices:", indices)

Requests indices: [0, 7, 14, 27, 30, 35, 42, 51, 57, 61, 65, 69, 77, 97, 106, 110, 111, 127, 131, 132, 134, 137, 143, 145, 146, 150, 174, 182, 191, 207, 215, 223, 235, 253, 255, 256, 267, 273, 286, 288, 298, 304, 313, 314, 315, 318, 325, 327, 329, 345, 352, 353, 356, 386, 401, 407, 420, 426, 444, 448, 450, 456, 458, 459, 464, 471, 473, 476, 487, 489, 494, 499, 513, 518, 523, 524, 529, 536, 544, 547, 559, 560, 581, 582, 585, 586, 589, 591, 597, 618, 620, 621, 624, 634, 650, 660, 669, 670, 671, 672, 677, 682, 685, 688, 689, 696, 698, 709, 710, 714, 723, 726, 728, 739, 745, 747, 757, 771, 792, 794, 796, 801, 805, 844, 848, 859, 862, 876, 892, 901, 909, 910, 914, 918, 934, 946, 960, 965, 980, 982, 986, 992, 999, 1003, 1012, 1013, 1020]


In [25]:
index_desired = 57
#index_desired = np.argmax(repertoire.fitnesses)
print(f"Index desired: {index_desired}, Max Fitness: {repertoire.fitnesses[index_desired]}")

Index desired: 57, Max Fitness: 2037.2403564453125


In [29]:
#index_desired = jnp.argmax(repertoire.fitnesses)
policy_params = jax.tree_util.tree_map(lambda x: x[index_desired], repertoire.genotypes)


random_key, subkey = jax.random.split(random_key)
state = reset_fn(subkey)
rollout = [state]
while True:
    _, actions = policy_network.apply(policy_params, state.obs)
    state = step_fn(state, actions)

    if state.done:
        break
    else:
        rollout.append(state)

if env.state_descriptor_name == "feet_contact":
    descriptor = sum([s.info["state_descriptor"] for s in rollout]) / len(rollout)
elif env.state_descriptor_name == "xy_position":
    descriptor = rollout[-1].info["state_descriptor"]
else:
    raise NotImplementedError

print("Episode length: {}/{}".format(len(rollout), config.env.episode_length))
print("Fitness: {}".format(sum([s.reward for s in rollout])))
print("Observed descriptor: {}".format(descriptor))

Episode length: 103/1000
Fitness: 169.89430236816406
Observed descriptor: [0.17475729]


In [39]:
state.obs.shape

(18,)

In [19]:
a = html.render(env.sys, [state.qp for state in rollout])
with open("rollout.html", "w") as f:
    f.write(a)
    


HTML(a)